In [1]:
# nuclio: ignore
import nuclio

In [3]:
%run set_env.ipynb

In [4]:
import os
import sys
import time

In [5]:
def remove(path):
    try:
     if os.path.exists(path):
        os.remove(path)
    except OSError:
        print ("Unable to remove file: %s"%path)

In [6]:
def cleanup(context,number_of_hours, path):
    """
    Removes files from the passed in path that are older than or equal 
    to the number_of_days
    """
    time_in_secs = time.time() - (number_of_hours * 60)
    for root, dirs, files in os.walk(path, topdown=False):
        for file_ in files:
            full_path = os.path.join(root, file_)
            stat = os.stat(full_path)
            
            if stat.st_mtime <= time_in_secs:
                #context.logger.info("Removing %s"%full_path )
                remove(full_path)
            else:
                context.logger.info("Not Removing %s"%full_path )
            
        #if not os.listdir(root):
        #    remove(root)

In [7]:
def handler(context,event):
    cleanup(context,int(os.getenv('HOURS',1)), os.path.join(os.getenv('NETAPP_MOUNT_PATH'),os.getenv('FEATURES_TABLE') ) )
    return ''

In [8]:
# nuclio: ignore
spec = nuclio.ConfigSpec(config={"spec.triggers.inference.kind":"cron",
                                "spec.triggers.inference.attributes.interval" :"5m",
                                "spec.readinessTimeoutSeconds" : 360 },
                         env={'FEATURES_TABLE': FEATURES_TABLE,
                              'NETAPP_MOUNT_PATH' : NETAPP_MOUNT_PATH,
                              'HOURS' : 6
                             },
                        ).with_v3io().add_volume(IGZ_MOUNT_PATH,IGZ_CONTAINER_PATH,kind='v3io',name='v3io')
if not NETAPP_SIM:
    spec.add_volume(NETAPP_MOUNT_PATH,NETAPP_PVC_CLAIM,kind='pvc',name='netapp')

In [ ]:
# nuclio: ignore
addr = nuclio.deploy_file(source=os.path.join(APP_DIR,'cleanup.ipynb'),
                                                  name='cleanup',project="netops",verbose=True, spec=spec, tag='v1.1')